In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define a ticker and a date range for your data
ticker = 'SPY'  # S&P 500 ETF, a good proxy for the market
start_date = '2000-01-01'
end_date = '2025-07-31'

# Download historical data from Yahoo Finance
# We will explicitly use auto_adjust=True to get the adjusted closing prices.
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

display(data)


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2000-01-03,92.142517,93.924388,91.152589,93.924388,8164300
2000-01-04,88.539223,91.271426,88.469928,90.934850,8089800
2000-01-05,88.697601,89.667731,86.955327,88.658004,12177900
2000-01-06,87.272079,89.647907,87.272079,88.459993,6227200
2000-01-07,92.340515,92.340515,88.737176,88.895565,8066500
...,...,...,...,...,...
2025-07-24,634.419983,636.150024,633.989990,634.599976,71307100
2025-07-25,637.099976,637.580017,634.840027,635.090027,56865400
